In [ ]:
# 1. Install Dependencies
!pip install streamlit pyngrok requests pillow

In [ ]:
# 2. Install & Start Ollama (Local LLM Backend)
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess
import time

# Start Ollama in the background
subprocess.Popen(["ollama", "serve"])
time.sleep(5) # Wait for it to start

In [ ]:
# 3. Pull Vision Model (This may take a few minutes)
# We use 'llava' as it is lightweight and reliable on Colab. 
# You can change this to 'qwen2.5-vl' if you want better performance but slower download.
!ollama pull llava

In [ ]:
# 4. Create the App File (app.py)
app_code = """
import streamlit as st
import requests
import json
import os
import uuid
from datetime import datetime

# --- Config ---
OLLAMA_URL = 'http://localhost:11434'
DEFAULT_MODEL = 'llava'

st.set_page_config(page_title='DSE AI Tutor (Colab)', page_icon='🎓', layout='wide')

def call_ollama_vision(model, image_bytes, prompt):
    url = f'{OLLAMA_URL}/api/generate'
    import base64
    img_b64 = base64.b64encode(image_bytes).decode('utf-8')
    payload = {
        'model': model,
        'prompt': prompt,
        'images': [img_b64],
        'stream': False
    }
    try:
        response = requests.post(url, json=payload, timeout=180)
        return response.json().get('response', '')
    except Exception as e:
        return f'Error: {str(e)}'

# --- UI ---
st.title('🎓 DSE AI Tutor (Cloud Version)')
st.info('Running on Google Colab GPU')

if 'messages' not in st.session_state:
    st.session_state.messages = []

for msg in st.session_state.messages:
    with st.chat_message(msg['role']):
        st.markdown(msg['content'])
        if 'image' in msg:
            st.image(msg['image'], width=300)

# Input
with st.popover('📎 Attach Image'):
    uploaded_file = st.file_uploader('Upload', type=['jpg','png'])

if uploaded_file:
    st.image(uploaded_file, width=150, caption='Preview')

user_input = st.chat_input('Ask a question...')

if user_input:
    # User Message
    with st.chat_message('user'):
        st.markdown(user_input)
        if uploaded_file:
            st.image(uploaded_file, width=300)
    
    msg_data = {'role': 'user', 'content': user_input}
    
    # Logic
    response_text = ''
    if uploaded_file:
        with st.spinner('👀 VLM Analyzing...'):
            img_bytes = uploaded_file.getvalue()
            prompt = f'You are a DSE Computer Science Tutor. {user_input}'
            response_text = call_ollama_vision(DEFAULT_MODEL, img_bytes, prompt)
            msg_data['image'] = img_bytes
    else:
        # Simple text fallback since AnythingLLM is not in Colab
        response_text = '⚠️ Text-only RAG is not available in this Colab demo (requires local AnythingLLM). Please upload an image for VLM analysis.'
    
    st.session_state.messages.append(msg_data)
    st.session_state.messages.append({'role': 'assistant', 'content': response_text})
    
    with st.chat_message('assistant'):
        st.markdown(response_text)
"""

with open("app.py", "w") as f:
    f.write(app_code)

In [ ]:
# 5. Run Streamlit with Ngrok Tunnel
from pyngrok import ngrok

# Terminate open tunnels if any
ngrok.kill()

# Set your authtoken (Optional but recommended for stability)
# ngrok.set_auth_token("YOUR_NGROK_TOKEN") 

# Open a HTTP tunnel on port 8501
public_url = ngrok.connect(8501).public_url
print(f"🚀 App is live at: {public_url}")

# Run Streamlit
!streamlit run app.py